### Thresh algorithm

In [1]:
import numpy as np

In [2]:
def Ber(p, size=1):
    return np.random.binomial(1,p, size=size)

In [3]:
n = 1e5 # num users
T = 1e2 # num epochs
L = 1e3 # min subgroup size
m = 1e2 # num subgroups
l = 10  # epochs length
eps = 0.01 # privacy parameter
delta = 1e-5 # failure parameter

In [28]:
class Users():
    def __init__(self, T, l, a, b, eps, tau, n):
        self.a = a
        self.b = b
        self.T = T
        self.eps = eps
        self.l = l
        self.cV = np.zeros((n,))
        self.cE = np.zeros((n,))
        self.tau = tau
        self.n = n
        self.xi = np.random.randint(0,2, size=n*l).reshape(n, l)
        
    def local_estimate(self):
        return np.mean(self.xi[:,-self.l:], axis=1)
    
    def vote(self, p_global, t):
        p_local = self.local_estimate()
        
        log_T = int(np.floor(np.log(T)))
        b_star = np.c_[[self.tau(b)*((p_local - p_global) > self.tau(b)) for b in range(-1, log_T)]].max(axis=0)
        
        VoteYes = (self.cV < self.eps/4) & np.logical_not(t % 2**(log_T - b_star))
        
        at = Ber(1 / (np.exp(self.a) + 1), size=self.n)
        at[VoteYes] = Ber(np.exp(self.a) / (np.exp(self.a) + 1), size=VoteYes.sum())
        return at
    
    def est(self, t):
        SendEstimate = self.cE < self.eps/4
        self.cE += self.b * SendEstimate
        
        p_t = Ber(1/(np.exp(b) + 1), size=self.n)
        p_t[SendEstimate] = Ber(
            (1 + self.local_estimate()*(np.exp(self.b)-1)) /\
            (np.exp(self.b) + 1), size=SendEstimate.sum()
        )
        return p_t
    
    def fake_est(self, t):
        return Ber(1 / (np.exp(self.b) + 1), size=self.n)

class Aggregator():
    def __init__(self, eps, delta, num_epochs, epoch_length, num_users, min_subgroup_size):
        self.eps = eps
        self.delta = delta
        self.T = num_epochs
        self.l = epoch_length
        self.n = num_users
        self.L = min_subgroup_size
        self.m = self.n // self.L
        tmp0 = np.log(12*self.m*self.T / delta)
        self.a = 4 * np.sqrt(2 * self.n * tmp0) /\
                 (self.L - 3/np.sqrt(2)*np.sqrt(self.n * tmp0))
        # vote noise level
        tmp1 = np.log(12*self.T / delta) / 2
        self.b = np.sqrt(2 * tmp1/self.n) /\
                 (np.log(self.T)*np.sqrt(tmp1 / self.l) - np.sqrt(tmp1 / self.n))
        # estimate noise level
        
        self.last_pub = -1
        self.p_last = -1
        
    def init_users(self):
        def tau(b):
            return 2 * (b + 1) * np.sqrt(np.log(10 * self.n * self.T / self.delta) / (2*self.l))
        return Users(self.T, self.l, self.a, self.b, self.eps, tau, self.n)
    
    def epoch(self, t, users):
        ats = users.vote(self.p_last, t)
        
        print(np.mean(ats))
        
        val = ( 1/(np.exp(self.a) + 1) + np.sqrt(np.log(10*self.T/self.delta)/(2*self.n)))
        print(val)
        global_update = (
            np.mean(ats) > val
        )
        
        
        if global_update:
            self.last_pub = t
            pts = users.est(t)
            self.p_last = np.mean( (p_ti*(np.exp(b)+1) - 1) / (np.exp(b) - 1) )
        else:
            pts = users.fake_est(t)
        return self.p_last

In [38]:
agg = Aggregator(1, 1e-6, 10**3, 100, 10**6, 100)

In [39]:
usrs = agg.init_users()

In [40]:
agg.epoch(1, usrs)

0.936799
0.9397632884135295


-1

In [48]:
usrs.vote(0.5, 1).mean()

0.936517

In [ ]:
class User():
    def __init__(self, T, l, a, b, eps, tau):
        self.a = a
        self.b = b
        self.T = T
        self.eps = eps
        self.xi = []
        self.l = l
        self.cV = 0
        self.cE = 0
        self.tau = tau
        
    def add_data(self, data=0):
        self.xi.append(data)
        
    def local_estimate(self):
        return np.mean(self.xi[-self.l:])
    
    def vote(self, p_global, t):
        p_local = self.local_estimate()
        
        b_star = max([b for b in range(-1, int(np.floor(np.log(T))))\
                     if np.abs(p_local - p_global) > self.tau(b)])
        
        VoteYes = (self.cV < self.eps/4) and not (t % 2**(np.floor(np.log(T)) - b_star))
        
        # print(VoteYes)
        if VoteYes:
            self.cV = self.cV + self.a # why would we add a?
            at = Ber(np.exp(self.a) / (np.exp(self.a) + 1))
        else:
            at = Ber(1 / (np.exp(self.a) + 1))
        return at
    
    def est(self, t):
        SendEstimate = self.cE < self.eps/4
        if SendEstimate:
            self.cE = self.cE + self.b
            p_t = Ber(
                (1 + self.local_estimate()*(np.exp(self.b)-1)) /\
                (np.exp(self.b) + 1)
            )
        else:
            p_t = Ber(
                1 / (np.exp(b) + 1)
            )
        return p_t
    
    def fake_est(self, t):
        return Ber(1 / (np.exp(self.b) + 1))